# Import Packages

In [1]:
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)
import numpy as np
from itertools import product

# Parameters

In [2]:
mini_lat_len_for_rank = 1000
max_prod_threshold_period = 6

# Load Files

In [3]:
vt_data = pd.read_csv('C:/Users/91709/Downloads/Work/UpWork/Andrew/Raw Arc Data/Vt_Wells_On_Section_With_Hz_Wells_Export.csv')
hz_data = pd.read_csv('C:/Users/91709/Downloads/Work/UpWork/Andrew/Raw Arc Data/Directional_Stick_Sections_Intersect_Export.csv')
vt_prod = pd.read_csv('C:/Users/91709/Downloads/Work/UpWork/Andrew/Raw DI Data/OK Blaine Custer Ellis Production Producing Entity Monthly Production.csv')
vt_header = pd.read_csv('C:/Users/91709/Downloads/Work/UpWork/Andrew/Raw DI Data/OK Blaine Custer Ellis Production Production Headers.csv')
hz_prod = pd.read_csv('C:/Users/91709/Downloads/Work/UpWork/Andrew/Raw DI Data/OK Blaine Custer Ellis Spud 2020+ Producing Entity Monthly Production.csv')
hz_well_header = pd.read_csv('C:/Users/91709/Downloads/Work/UpWork/Andrew/Raw DI Data/OK Blailne Custer Ellis Hz Spud 202+ Well Headers.csv')
hz_header = pd.read_csv('C:/Users/91709/Downloads/Work/UpWork/Andrew/Raw DI Data/OK Blaine Custer Ellis Spud 2020+ Production Headers.csv')

# Data Warangling

Vt Prod

In [4]:
vt_prod['Monthly Production Date'] = pd.to_datetime(vt_prod['Monthly Production Date'])
vt_prod = vt_prod[(vt_prod['Monthly Production Date']> "2016-12-31")]

vt_data_prod = vt_data[['APIUWI', 'DrillType', 'ok_blm_p_5']]
vt_prod = vt_prod.merge(vt_data_prod, how='left', left_on=["API/UWI"], right_on=["APIUWI"])

,Entity ID,API/UWI,API/UWI List,Monthly Production Date,Monthly Oil,Monthly Gas,Monthly Water,Well Count,Days,Daily Avg Oil,...,Well/Lease Name,Well Number,Operator Company Name,Production Type,Production Status,Entity Type,Producing Month Number,APIUWI,DrillType,ok_blm_p_5
0,130963218,35011200000000.00,3501124262.00,2021-01-08,4654.00,140613.00,0.00,1,nan,150.13,...,EDGEMERE FARMS,3-25-36XHM,CONTINENTAL RESOURCES,O&G,ACTIVE,WELL,1,35011200000000.00,H,32-T14N-R13W
1,130963218,35011200000000.00,3501124262.00,2021-01-08,4654.00,140613.00,0.00,1,nan,150.13,...,EDGEMERE FARMS,3-25-36XHM,CONTINENTAL RESOURCES,O&G,ACTIVE,WELL,1,35011200000000.00,D,28-T14N-R13W
2,130963218,35011200000000.00,3501124262.00,2021-01-08,4654.00,140613.00,0.00,1,nan,150.13,...,EDGEMERE FARMS,3-25-36XHM,CONTINENTAL RESOURCES,O&G,ACTIVE,WELL,1,35011200000000.00,H,28-T14N-R13W
3,130963218,35011200000000.00,3501124262.00,2021-01-08,4654.00,140613.00,0.00,1,nan,150.13,...,EDGEMERE FARMS,3-25-36XHM,CONTINENTAL RESOURCES,O&G,ACTIVE,WELL,1,35011200000000.00,H,20-T14N-R13W
4,130963218,35011200000000.00,3501124262.00,2021-01-08,4654.00,140613.00,0.00,1,nan,150.13,...,EDGEMERE FARMS,3-25-36XHM,CONTINENTAL RESOURCES,O&G,ACTIVE,WELL,1,35011200000000.00,H,12-T14N-R13W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9338101,103708042,0.00,0.00,2021-01-08,164.00,nan,nan,1,nan,5.29,...,CHRISTENSEN B #1,NaN,SOUTHERN RESOURCES INC,OIL,ACTIVE,LEASE,1,0.00,U,2-T13N-R14W
9338102,103708042,0.00,0.00,2021-01-08,164.00,nan,nan,1,nan,5.29,...,CHRISTENSEN B #1,NaN,SOUTHERN RESOURCES INC,OIL,ACTIVE,LEASE,1,0.00,U,1-T14N-R13W
9338103,103708042,0.00,0.00,2021-01-08,164.00,nan,nan,1,nan,5.29,...,CHRISTENSEN B #1,NaN,SOUTHERN RESOURCES INC,OIL,ACTIVE,LEASE,1,0.00,U,25-T15N-R10W
9338104,103708042,0.00,0.00,2021-01-08,164.00,nan,nan,1,nan,5.29,...,CHRISTENSEN B #1,NaN,SOUTHERN RESOURCES INC,OIL,ACTIVE,LEASE,1,0.00,U,11-T15N-R10W


HzAPI Unique & HzSection Unique             

In [5]:
hzapi_unique = hz_data[['UID01', 'API14']]
hzapi_unique = hzapi_unique.drop_duplicates(keep = 'first')

hzsection_unique = hz_data[['UID01_1', 'section']]
hzsection_unique = hzsection_unique.drop_duplicates(keep = 'first')

Lat Lenght

In [6]:
lat_lenght = hz_data.groupby(['API14', 'section'])['LEN02'].sum().ffill().reset_index()

Section Rank

In [7]:
section_rank = lat_lenght[(lat_lenght['LEN02']>= mini_lat_len_for_rank)]

section_rank["rank"] = section_rank.groupby("API14")["LEN02"].rank(ascending=0,method='min')

Section Spud

In [8]:
section_spud = section_rank[['API14', 'section']]
hz_well_header_sec_spud = hz_well_header[['API14','Spud Date']]

section_spud['API14'] = section_spud['API14'].astype(str)
hz_well_header_sec_spud['API14'] = hz_well_header_sec_spud['API14'].astype(str)

section_spud = section_spud.merge(hz_well_header_sec_spud, how='left', on=["API14"])

Sections

In [9]:
sections = section_spud.groupby('section').agg({'Spud Date': ['min']}).reset_index()
sections.columns = sections.columns.get_level_values(0)
sections = sections.merge(section_spud, how='left', on=["section", "Spud Date"])

Vt Header

In [10]:
vt_data_header = vt_data[['APIUWI', 'ok_blm_p_5']]
section_header = sections[['section', 'Spud Date']]

vt_header = vt_header.merge(vt_data_header, how='left', left_on=["API/UWI"], right_on=["APIUWI"])
vt_header = vt_header.merge(section_header, how='left', left_on=["ok_blm_p_5"], right_on=["section"])
vt_header.rename(columns = {'Spud Date_y':'Min Spud Data'}, inplace = True)


Prod Oil

In [13]:
comb_list = list(product(hzapi_unique['API14'], hzsection_unique['section']))

prod_oil = pd.DataFrame(data=comb_list, columns=['API14','section'])
well_header_oil = hz_well_header[['API14', 'Spud Date']]
vt_prod_oil = vt_prod[['ok_blm_p_5', 'Monthly Production Date', 'Monthly Oil']]

prod_oil = prod_oil.merge(well_header_oil, how='left', on=['API14'])

prod_oil['Spud Date'] = pd.to_datetime(prod_oil['Spud Date'])
prod_oil['Start Date'] =  prod_oil['Spud Date'].to_numpy().astype('datetime64[M]')
prod_oil['Start Date'] = prod_oil['Start Date'] + pd.offsets.MonthOffset(-max_prod_threshold_period)



,API14,section,Spud Date,Start Date
0,35045238430000,19-T19N-R25W,2021-09-21,2021-03-01
1,35045238430000,18-T19N-R25W,2021-09-21,2021-03-01
2,35045238430000,5-T17N-R24W,2021-09-21,2021-03-01
3,35045238430000,32-T18N-R24W,2021-09-21,2021-03-01
4,35045238430000,29-T18N-R24W,2021-09-21,2021-03-01
...,...,...,...,...
6009,35011242300000,11-T15N-R10W,2021-10-27,2021-04-01
6010,35011242300000,14-T15N-R10W,2021-10-27,2021-04-01
6011,35011242300000,23-T15N-R10W,2021-10-27,2021-04-01
6012,35011242300000,21-T15N-R10W,2021-10-27,2021-04-01


In [19]:
start_date = prod_oil['Start Date'].to_list()
end_date = prod_oil['Spud Date'].to_list()
section = prod_oil['section'].to_list()

In [ ]:
res = pd.DataFrame()

for (s,e,c) in zip(start_date, end_date, section):
    sumif = vt_prod_oil[(vt_prod_oil['ok_blm_p_5'] == c) & \
         (vt_prod_oil['Monthly Production Date']>= s) & (vt_prod_oil['Monthly Production Date']<= e)]
    val = sumif.sum()
    new_line =  pd.DataFrame({'Oil Production': val}, index=[0])
    res = res.append(new_line)
    val.empty

In [ ]:
prod_oil['Oil Production']= ''

for i1, p1 in prod_oil.iterrows():
    start_date = p1['Start Month']
    start_date = pd.to_datetime(start_date)
    end_date = p1['Spud Date']
    end_date = pd.to_datetime(end_date)
    tmp_df = vt_prod_oil[(vt_prod_oil['ok_blm_p_5'] == p1["section"]) & \
         (vt_prod_oil['Monthly Production Date']>= start_date) & (vt_prod_oil['Monthly Production Date']<= end_date)]               
    prod_oil["Oil Production"][i1]= sum(tmp_df["Monthly Oil"])

In [ ]:
prod_oil.dtypes